## Purpose: Try different models-- Part6.
### forest

In [1]:
# import dependencies.
import pandas as pd
import numpy as np

from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

#### STEP1: Read in dataset.  Remove data from 2016-2019.
- data from 2016-2018 will be used to bs test the model.
- data from 2019 will be used to predict the winners of the 2019 WS.

In [2]:
# read in the data.
team_data = pd.read_csv("../../Resources/clean_data_1905.csv")
del team_data["Unnamed: 0"]
team_data.head()

,team,year,A,DP,E,G2,GS2,INN,PB,PO,...,R1,SHO,SO1,SV,TBF,W,WHIP,WP,WPCT,winners
0,St. Louis Cardinals,2019,1033,114,43,104,936,8313.0,3,2771,...,456,4,895,33,3896,56,1.29,21,0.538,0
1,Arizona Diamondbacks,2019,1010,83,45,105,945,8538.0,2,2846,...,472,7,925,24,4001,53,1.28,35,0.505,0
2,Kansas City Royals,2019,990,105,45,106,954,8421.0,6,2807,...,543,5,816,24,4125,39,1.46,34,0.368,0
3,Houston Astros,2019,875,54,50,106,954,8589.0,6,2863,...,432,7,1074,27,3929,67,1.14,31,0.632,0
4,Tampa Bay Rays,2019,975,92,53,107,963,8760.0,11,2920,...,409,6,1037,26,3985,59,1.16,40,0.551,0


In [3]:
# remove data from 2016 through 2019.
team_data_new = team_data.loc[team_data["year"] < 2016]
team_data_new.head()

,team,year,A,DP,E,G2,GS2,INN,PB,PO,...,R1,SHO,SO1,SV,TBF,W,WHIP,WP,WPCT,winners
120,San Francisco Giants,2015,1639,136,72,162,1458,13143.0,6,4381,...,631,11,1309,43,6048,87,1.21,40,0.537,0
121,Washington Nationals,2015,1425,142,73,162,1458,13137.0,17,4379,...,612,12,1476,46,6036,95,1.19,47,0.586,0
122,Houston Astros,2015,1599,135,77,162,1458,13212.0,18,4404,...,701,8,1396,44,6180,84,1.29,98,0.519,0
123,Detroit Tigers,2015,1537,148,75,161,1449,12852.0,5,4284,...,721,8,1232,47,6048,86,1.32,44,0.534,0
124,Boston Red Sox,2015,1427,139,75,162,1458,12957.0,37,4319,...,694,5,1362,43,6073,93,1.27,52,0.574,0


In [4]:
target = team_data_new["winners"]
features = team_data_new.drop({"team", "year", "winners"}, axis=1)
feature_columns = list(features.columns)
print (target.shape)
print (features.shape)
print (feature_columns)

(2344,)
(2344, 52)
['A', 'DP', 'E', 'G2', 'GS2', 'INN', 'PB', 'PO', 'TC', '2B', '3B', 'AB', 'AO', 'BB', 'G', 'H', 'HBP', 'HR', 'NP_x', 'OBP', 'OPS_x', 'PA', 'R', 'RBI', 'SAC', 'SB', 'SLG', 'TB', 'XBH', 'BB1', 'BK', 'CG', 'ER', 'ERA', 'G1', 'GF', 'GS', 'H1', 'HB', 'HR1', 'IP', 'L', 'OBP1', 'R1', 'SHO', 'SO1', 'SV', 'TBF', 'W', 'WHIP', 'WP', 'WPCT']


#### STEP2: Split data. Train and test the model.

In [5]:
def tree(features, target):
    '''
    INPUT: 
    -features = dataset without team names and year.
    -target = labels.
    
    OUTPUT:
    -model = model output
    
    DESCRIPTION:
    -dataset is taken in and split into test and train sets.
    -model is fit
    -model is tested
    '''
    
    # split data.
    X_train, X_test, y_train, y_test = train_test_split(features, target)

    # train model.
    model = RandomForestClassifier(n_jobs=-1,
                                   n_estimators=100)
    model.fit(X_train, y_train)

    # scale data.
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.fit_transform(X_test)
    
    # predict.
    prediction = model.predict(X_test)

    print (classification_report(y_test, prediction, target_names=["0", "1"]))
    return model

In [6]:
model = tree(features, target)

              precision    recall  f1-score   support

           0       0.94      1.00      0.97       549
           1       0.00      0.00      0.00        37

   micro avg       0.94      0.94      0.94       586
   macro avg       0.47      0.50      0.48       586
weighted avg       0.88      0.94      0.91       586



/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Applications/anacon

This is terrible.  Let's try upsampling.

#### STEP3: Upsampling

In [7]:
# upsample for a more balanced dataset.
def upsample(dataset, no_samples):
    '''
    INPUT: 
    -dataset = dataset without team names and year.
    -n_samples = number of minority_unsampled.
    
    OUTPUT:
    -X_train_scaled = scaled X train data.
    -X_test_scaled = scaled X test data.
    -y_train = y train data
    -y_test = y test data
    
    DESCRIPTION:
    -dataset is taken in and split into minority and majority classes.
    -dataset is then upsampled for the mainority class
    -split the data into features and targets
    -split data into train and test sets
    -train and test sets were are scaled.
    '''
    
    # separate majority and minority classes.
    df_majority = dataset.loc[dataset["winners"] == 0]
    df_minority = dataset.loc[dataset["winners"] == 1]

    # upsample minority class.
    df_minority_unsampled = resample(df_minority,
                                    replace=True,
                                    n_samples=no_samples,
                                    random_state=123)

    # combine majority class with upsampled minority class.
    df_upsampled = pd.concat([df_majority, df_minority_unsampled])

    # separate features and target.
    y = df_upsampled["winners"]
    X = df_upsampled[feature_columns]
    
    model = tree(X, y)
    return model

In [8]:
model_100 = upsample(team_data_new, 2234)
print ("_______________________________")
model_50 = upsample(team_data_new, 1117)
print ("_______________________________")
model_25 = upsample(team_data_new, 559)
print ("_______________________________")
model_12 = upsample(team_data_new, 250)
print ("_______________________________")
model_6 = upsample(team_data_new, 200)
print ("_______________________________")
model_3 = upsample(team_data_new, 150)


/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


              precision    recall  f1-score   support

           0       1.00      0.99      1.00       535
           1       0.99      1.00      1.00       582

   micro avg       1.00      1.00      1.00      1117
   macro avg       1.00      1.00      1.00      1117
weighted avg       1.00      1.00      1.00      1117

_______________________________


/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


              precision    recall  f1-score   support

           0       1.00      0.99      1.00       540
           1       0.99      1.00      0.99       298

   micro avg       1.00      1.00      1.00       838
   macro avg       1.00      1.00      1.00       838
weighted avg       1.00      1.00      1.00       838

_______________________________


/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


...

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


              precision    recall  f1-score   support

           0       0.98      1.00      0.99       556
           1       1.00      0.78      0.88        65

   micro avg       0.98      0.98      0.98       621
   macro avg       0.99      0.89      0.93       621
weighted avg       0.98      0.98      0.98       621

_______________________________


/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


              precision    recall  f1-score   support

           0       0.96      1.00      0.98       552
           1       0.97      0.65      0.78        57

   micro avg       0.97      0.97      0.97       609
   macro avg       0.97      0.82      0.88       609
weighted avg       0.97      0.97      0.96       609

_______________________________
              precision    recall  f1-score   support

           0       0.97      1.00      0.98       555
           1       1.00      0.51      0.68        41

   micro avg       0.97      0.97      0.97       596
   macro avg       0.98      0.76      0.83       596
weighted avg       0.97      0.97      0.96       596



/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


Probably overfit.

In [9]:
def predict_the_winner(model, year, team_data):
    '''
    INPUT: 
    -X_train = scaled X train data.
    -model = the saved model.
    -team_data = complete dataframe with all data.
    -year = the year want to look at.
    
    OUTPUT:
    -printed prediction.
    
    DESCRIPTION:
    -data from year of interest is isolated.
    -the data are scaled.
    -the prediction is made.
    -print out the resulting probability and the name of the team.
    '''
    
    # grab the data.
    team_data = team_data.loc[team_data["year"] == year].reset_index()

    # set features (no team, year, winners).
    # set target (winners).
    features = team_data[feature_columns]
    
    # fit the model.
    probabilities = model.predict_proba(features)

    # convert predictions to datafram.e
    WS_predictions = pd.DataFrame(probabilities[:,1])

    # Sort the DataFrame (descending)
    WS_predictions = WS_predictions.sort_values(0, ascending=False)

    WS_predictions['Probability'] = WS_predictions[0]

    # Print 50 highest probability HoF inductees from still eligible players
    for i, row in WS_predictions.head(50).iterrows():
       prob = ' '.join(('WS Probability =', str(row['Probability'])))
       print('')
       print(prob)
       print(team_data.iloc[i,1:27]["team"])

In [10]:
predict_the_winner(model_100, 2018, team_data)


WS Probability = 0.21
Philadelphia Phillies

WS Probability = 0.19
Texas Rangers

WS Probability = 0.19
Atlanta Braves

WS Probability = 0.18
Los Angeles Angels

WS Probability = 0.17
Milwaukee Brewers

WS Probability = 0.16
Boston Red Sox

WS Probability = 0.15
Washington Nationals

WS Probability = 0.15
St. Louis Cardinals

WS Probability = 0.15
Kansas City Royals

WS Probability = 0.14
New York Yankees

WS Probability = 0.13
Houston Astros

WS Probability = 0.13
New York Mets

WS Probability = 0.13
Pittsburgh Pirates

WS Probability = 0.12
Oakland Athletics

WS Probability = 0.12
Colorado Rockies

WS Probability = 0.11
Seattle Mariners

WS Probability = 0.11
Chicago White Sox

WS Probability = 0.11
Baltimore Orioles

WS Probability = 0.11
Tampa Bay Rays

WS Probability = 0.11
Minnesota Twins

WS Probability = 0.1
Detroit Tigers

WS Probability = 0.1
San Diego Padres

WS Probability = 0.1
Chicago Cubs

WS Probability = 0.1
San Francisco Giants

WS Probability = 0.1
Arizona Diamondba

In [11]:
predict_the_winner(model_50, 2018, team_data)


WS Probability = 0.25
Los Angeles Angels

WS Probability = 0.22
Philadelphia Phillies

WS Probability = 0.22
Texas Rangers

WS Probability = 0.21
Boston Red Sox

WS Probability = 0.2
Milwaukee Brewers

WS Probability = 0.2
Arizona Diamondbacks

WS Probability = 0.19
Oakland Athletics

WS Probability = 0.19
Pittsburgh Pirates

WS Probability = 0.18
San Francisco Giants

WS Probability = 0.17
Baltimore Orioles

WS Probability = 0.17
Atlanta Braves

WS Probability = 0.17
Toronto Blue Jays

WS Probability = 0.17
Kansas City Royals

WS Probability = 0.17
Washington Nationals

WS Probability = 0.16
New York Yankees

WS Probability = 0.16
Chicago White Sox

WS Probability = 0.16
Minnesota Twins

WS Probability = 0.16
Seattle Mariners

WS Probability = 0.16
Houston Astros

WS Probability = 0.15
Detroit Tigers

WS Probability = 0.15
Tampa Bay Rays

WS Probability = 0.14
Cleveland Indians

WS Probability = 0.14
New York Mets

WS Probability = 0.13
Colorado Rockies

WS Probability = 0.12
Chicago

In [12]:
predict_the_winner(model_25, 2018, team_data)


WS Probability = 0.35
Texas Rangers

WS Probability = 0.34
Los Angeles Angels

WS Probability = 0.33
Philadelphia Phillies

WS Probability = 0.33
New York Mets

WS Probability = 0.31
Milwaukee Brewers

WS Probability = 0.31
Pittsburgh Pirates

WS Probability = 0.29
Atlanta Braves

WS Probability = 0.29
Detroit Tigers

WS Probability = 0.28
Arizona Diamondbacks

WS Probability = 0.28
Toronto Blue Jays

WS Probability = 0.28
Minnesota Twins

WS Probability = 0.28
San Francisco Giants

WS Probability = 0.28
Chicago White Sox

WS Probability = 0.28
Boston Red Sox

WS Probability = 0.27
San Diego Padres

WS Probability = 0.27
Washington Nationals

WS Probability = 0.26
Baltimore Orioles

WS Probability = 0.26
Seattle Mariners

WS Probability = 0.26
Oakland Athletics

WS Probability = 0.26
Houston Astros

WS Probability = 0.26
Kansas City Royals

WS Probability = 0.25
Miami Marlins

WS Probability = 0.23
Los Angeles Dodgers

WS Probability = 0.23
New York Yankees

WS Probability = 0.22
Colo

In [13]:
predict_the_winner(model_12, 2018, team_data)


WS Probability = 0.35
Arizona Diamondbacks

WS Probability = 0.33
Houston Astros

WS Probability = 0.33
Minnesota Twins

WS Probability = 0.32
New York Yankees

WS Probability = 0.32
Los Angeles Angels

WS Probability = 0.3
Boston Red Sox

WS Probability = 0.29
Philadelphia Phillies

WS Probability = 0.29
Oakland Athletics

WS Probability = 0.28
Texas Rangers

WS Probability = 0.27
Washington Nationals

WS Probability = 0.27
Atlanta Braves

WS Probability = 0.27
Cleveland Indians

WS Probability = 0.27
Milwaukee Brewers

WS Probability = 0.26
San Francisco Giants

WS Probability = 0.26
Seattle Mariners

WS Probability = 0.26
Tampa Bay Rays

WS Probability = 0.25
Pittsburgh Pirates

WS Probability = 0.25
Los Angeles Dodgers

WS Probability = 0.24
St. Louis Cardinals

WS Probability = 0.23
Cincinnati Reds

WS Probability = 0.23
New York Mets

WS Probability = 0.21
Miami Marlins

WS Probability = 0.21
San Diego Padres

WS Probability = 0.21
Baltimore Orioles

WS Probability = 0.2
Toronto

In [14]:
predict_the_winner(model_6, 2018, team_data)


WS Probability = 0.32
Los Angeles Dodgers

WS Probability = 0.31
Arizona Diamondbacks

WS Probability = 0.31
Los Angeles Angels

WS Probability = 0.3
Houston Astros

WS Probability = 0.3
Boston Red Sox

WS Probability = 0.27
St. Louis Cardinals

WS Probability = 0.27
New York Mets

WS Probability = 0.27
Oakland Athletics

WS Probability = 0.27
Minnesota Twins

WS Probability = 0.27
Texas Rangers

WS Probability = 0.27
Milwaukee Brewers

WS Probability = 0.27
Cincinnati Reds

WS Probability = 0.26
Pittsburgh Pirates

WS Probability = 0.26
Philadelphia Phillies

WS Probability = 0.26
Tampa Bay Rays

WS Probability = 0.26
Kansas City Royals

WS Probability = 0.25
San Francisco Giants

WS Probability = 0.25
San Diego Padres

WS Probability = 0.24
Toronto Blue Jays

WS Probability = 0.24
Miami Marlins

WS Probability = 0.24
Chicago White Sox

WS Probability = 0.23
Washington Nationals

WS Probability = 0.23
Detroit Tigers

WS Probability = 0.22
Cleveland Indians

WS Probability = 0.22
New 

In [15]:
predict_the_winner(model_3, 2018, team_data)


WS Probability = 0.21
Houston Astros

WS Probability = 0.21
Minnesota Twins

WS Probability = 0.2
New York Yankees

WS Probability = 0.17
Los Angeles Dodgers

WS Probability = 0.15
Boston Red Sox

WS Probability = 0.14
Tampa Bay Rays

WS Probability = 0.12
Los Angeles Angels

WS Probability = 0.11
Philadelphia Phillies

WS Probability = 0.11
Texas Rangers

WS Probability = 0.11
Seattle Mariners

WS Probability = 0.11
Cleveland Indians

WS Probability = 0.11
Oakland Athletics

WS Probability = 0.1
Milwaukee Brewers

WS Probability = 0.1
Atlanta Braves

WS Probability = 0.09
Detroit Tigers

WS Probability = 0.09
Cincinnati Reds

WS Probability = 0.09
Pittsburgh Pirates

WS Probability = 0.09
Washington Nationals

WS Probability = 0.09
Toronto Blue Jays

WS Probability = 0.08
San Francisco Giants

WS Probability = 0.08
Arizona Diamondbacks

WS Probability = 0.08
St. Louis Cardinals

WS Probability = 0.07
Miami Marlins

WS Probability = 0.07
Chicago Cubs

WS Probability = 0.07
Baltimore O

In [16]:
predict_the_winner(model_100, 2017, team_data)


WS Probability = 0.14
Washington Nationals

WS Probability = 0.13
Los Angeles Dodgers

WS Probability = 0.12
Minnesota Twins

WS Probability = 0.12
Cleveland Indians

WS Probability = 0.1
San Francisco Giants

WS Probability = 0.1
Arizona Diamondbacks

WS Probability = 0.09
Baltimore Orioles

WS Probability = 0.08
Pittsburgh Pirates

WS Probability = 0.07
Los Angeles Angels

WS Probability = 0.07
Tampa Bay Rays

WS Probability = 0.07
New York Mets

WS Probability = 0.06
Philadelphia Phillies

WS Probability = 0.06
Detroit Tigers

WS Probability = 0.06
New York Yankees

WS Probability = 0.06
Houston Astros

WS Probability = 0.06
Seattle Mariners

WS Probability = 0.06
Atlanta Braves

WS Probability = 0.05
Texas Rangers

WS Probability = 0.05
Colorado Rockies

WS Probability = 0.04
Chicago Cubs

WS Probability = 0.04
Toronto Blue Jays

WS Probability = 0.03
Oakland Athletics

WS Probability = 0.03
San Diego Padres

WS Probability = 0.03
Boston Red Sox

WS Probability = 0.03
Milwaukee Br

In [17]:
predict_the_winner(model_50, 2017, team_data)


WS Probability = 0.17
Washington Nationals

WS Probability = 0.16
Pittsburgh Pirates

WS Probability = 0.16
Seattle Mariners

WS Probability = 0.14
Tampa Bay Rays

WS Probability = 0.14
Arizona Diamondbacks

WS Probability = 0.14
Los Angeles Angels

WS Probability = 0.13
Los Angeles Dodgers

WS Probability = 0.13
Chicago Cubs

WS Probability = 0.11
Cleveland Indians

WS Probability = 0.1
Detroit Tigers

WS Probability = 0.1
Minnesota Twins

WS Probability = 0.09
Oakland Athletics

WS Probability = 0.09
New York Mets

WS Probability = 0.08
Boston Red Sox

WS Probability = 0.08
New York Yankees

WS Probability = 0.07
Atlanta Braves

WS Probability = 0.07
St. Louis Cardinals

WS Probability = 0.06
Milwaukee Brewers

WS Probability = 0.06
Philadelphia Phillies

WS Probability = 0.05
Chicago White Sox

WS Probability = 0.05
Baltimore Orioles

WS Probability = 0.05
San Francisco Giants

WS Probability = 0.04
Toronto Blue Jays

WS Probability = 0.04
San Diego Padres

WS Probability = 0.04
Co

In [18]:
predict_the_winner(model_25, 2017, team_data)


WS Probability = 0.2
Los Angeles Dodgers

WS Probability = 0.19
New York Yankees

WS Probability = 0.18
Baltimore Orioles

WS Probability = 0.17
Cleveland Indians

WS Probability = 0.17
Seattle Mariners

WS Probability = 0.16
Oakland Athletics

WS Probability = 0.15
Arizona Diamondbacks

WS Probability = 0.15
Washington Nationals

WS Probability = 0.15
Atlanta Braves

WS Probability = 0.14
Houston Astros

WS Probability = 0.12
Tampa Bay Rays

WS Probability = 0.12
New York Mets

WS Probability = 0.1
San Diego Padres

WS Probability = 0.1
Pittsburgh Pirates

WS Probability = 0.1
Los Angeles Angels

WS Probability = 0.09
Philadelphia Phillies

WS Probability = 0.08
Toronto Blue Jays

WS Probability = 0.08
San Francisco Giants

WS Probability = 0.08
St. Louis Cardinals

WS Probability = 0.07
Detroit Tigers

WS Probability = 0.07
Boston Red Sox

WS Probability = 0.07
Milwaukee Brewers

WS Probability = 0.07
Chicago White Sox

WS Probability = 0.06
Minnesota Twins

WS Probability = 0.04
Co

In [19]:
predict_the_winner(model_12, 2017, team_data)


WS Probability = 0.27
Boston Red Sox

WS Probability = 0.23
Houston Astros

WS Probability = 0.22
Cleveland Indians

WS Probability = 0.19
Washington Nationals

WS Probability = 0.17
New York Yankees

WS Probability = 0.15
Los Angeles Dodgers

WS Probability = 0.14
Milwaukee Brewers

WS Probability = 0.13
Minnesota Twins

WS Probability = 0.13
Detroit Tigers

WS Probability = 0.12
Atlanta Braves

WS Probability = 0.11
New York Mets

WS Probability = 0.11
Los Angeles Angels

WS Probability = 0.1
Seattle Mariners

WS Probability = 0.1
Oakland Athletics

WS Probability = 0.09
Arizona Diamondbacks

WS Probability = 0.07
Toronto Blue Jays

WS Probability = 0.06
Chicago White Sox

WS Probability = 0.06
Philadelphia Phillies

WS Probability = 0.06
Chicago Cubs

WS Probability = 0.06
Baltimore Orioles

WS Probability = 0.05
Tampa Bay Rays

WS Probability = 0.04
Pittsburgh Pirates

WS Probability = 0.04
Colorado Rockies

WS Probability = 0.04
San Diego Padres

WS Probability = 0.03
St. Louis C

In [20]:
predict_the_winner(model_6, 2017, team_data)


WS Probability = 0.31
Houston Astros

WS Probability = 0.29
New York Yankees

WS Probability = 0.28
Cleveland Indians

WS Probability = 0.27
Boston Red Sox

WS Probability = 0.21
Oakland Athletics

WS Probability = 0.17
Atlanta Braves

WS Probability = 0.16
Los Angeles Dodgers

WS Probability = 0.14
Washington Nationals

WS Probability = 0.12
Tampa Bay Rays

WS Probability = 0.11
Minnesota Twins

WS Probability = 0.11
New York Mets

WS Probability = 0.09
Los Angeles Angels

WS Probability = 0.08
Milwaukee Brewers

WS Probability = 0.08
Arizona Diamondbacks

WS Probability = 0.07
Detroit Tigers

WS Probability = 0.06
Seattle Mariners

WS Probability = 0.06
San Diego Padres

WS Probability = 0.05
Chicago Cubs

WS Probability = 0.05
St. Louis Cardinals

WS Probability = 0.04
Miami Marlins

WS Probability = 0.04
Toronto Blue Jays

WS Probability = 0.04
Baltimore Orioles

WS Probability = 0.04
Pittsburgh Pirates

WS Probability = 0.04
Colorado Rockies

WS Probability = 0.03
Chicago White S

In [21]:
predict_the_winner(model_3, 2017, team_data)


WS Probability = 0.26
Houston Astros

WS Probability = 0.23
Boston Red Sox

WS Probability = 0.22
New York Yankees

WS Probability = 0.19
Cleveland Indians

WS Probability = 0.19
Oakland Athletics

WS Probability = 0.16
Washington Nationals

WS Probability = 0.15
Seattle Mariners

WS Probability = 0.14
Los Angeles Angels

WS Probability = 0.14
Los Angeles Dodgers

WS Probability = 0.12
Atlanta Braves

WS Probability = 0.11
Milwaukee Brewers

WS Probability = 0.11
Pittsburgh Pirates

WS Probability = 0.11
Tampa Bay Rays

WS Probability = 0.11
New York Mets

WS Probability = 0.1
Detroit Tigers

WS Probability = 0.09
San Francisco Giants

WS Probability = 0.08
Arizona Diamondbacks

WS Probability = 0.08
St. Louis Cardinals

WS Probability = 0.07
Toronto Blue Jays

WS Probability = 0.06
Minnesota Twins

WS Probability = 0.05
Miami Marlins

WS Probability = 0.05
Baltimore Orioles

WS Probability = 0.05
Colorado Rockies

WS Probability = 0.05
Philadelphia Phillies

WS Probability = 0.04
San

In [22]:
predict_the_winner(model_100, 2016, team_data)


WS Probability = 0.18
Houston Astros

WS Probability = 0.13
New York Yankees

WS Probability = 0.13
Washington Nationals

WS Probability = 0.12
Baltimore Orioles

WS Probability = 0.12
Chicago Cubs

WS Probability = 0.12
Cleveland Indians

WS Probability = 0.12
Los Angeles Dodgers

WS Probability = 0.11
Boston Red Sox

WS Probability = 0.09
Seattle Mariners

WS Probability = 0.09
Toronto Blue Jays

WS Probability = 0.08
Minnesota Twins

WS Probability = 0.08
Texas Rangers

WS Probability = 0.08
Arizona Diamondbacks

WS Probability = 0.07
Oakland Athletics

WS Probability = 0.07
Los Angeles Angels

WS Probability = 0.06
Tampa Bay Rays

WS Probability = 0.06
Milwaukee Brewers

WS Probability = 0.05
Kansas City Royals

WS Probability = 0.05
San Diego Padres

WS Probability = 0.04
St. Louis Cardinals

WS Probability = 0.04
Colorado Rockies

WS Probability = 0.03
Detroit Tigers

WS Probability = 0.03
Miami Marlins

WS Probability = 0.01
Atlanta Braves

WS Probability = 0.01
Pittsburgh Pira

In [23]:
predict_the_winner(model_50, 2016, team_data)


WS Probability = 0.21
Houston Astros

WS Probability = 0.16
Washington Nationals

WS Probability = 0.15
Tampa Bay Rays

WS Probability = 0.15
Chicago Cubs

WS Probability = 0.13
Minnesota Twins

WS Probability = 0.11
New York Yankees

WS Probability = 0.11
Los Angeles Dodgers

WS Probability = 0.11
Milwaukee Brewers

WS Probability = 0.11
Seattle Mariners

WS Probability = 0.1
Cleveland Indians

WS Probability = 0.09
Los Angeles Angels

WS Probability = 0.09
Boston Red Sox

WS Probability = 0.08
Texas Rangers

WS Probability = 0.07
Arizona Diamondbacks

WS Probability = 0.07
Toronto Blue Jays

WS Probability = 0.07
Kansas City Royals

WS Probability = 0.06
Oakland Athletics

WS Probability = 0.05
San Diego Padres

WS Probability = 0.04
Miami Marlins

WS Probability = 0.03
Pittsburgh Pirates

WS Probability = 0.03
San Francisco Giants

WS Probability = 0.03
Baltimore Orioles

WS Probability = 0.02
New York Mets

WS Probability = 0.02
Colorado Rockies

WS Probability = 0.02
St. Louis Ca

In [24]:
predict_the_winner(model_25, 2016, team_data)


WS Probability = 0.27
Washington Nationals

WS Probability = 0.26
Los Angeles Dodgers

WS Probability = 0.25
Houston Astros

WS Probability = 0.18
New York Yankees

WS Probability = 0.17
Seattle Mariners

WS Probability = 0.16
Los Angeles Angels

WS Probability = 0.14
Tampa Bay Rays

WS Probability = 0.13
Boston Red Sox

WS Probability = 0.12
Chicago Cubs

WS Probability = 0.11
Minnesota Twins

WS Probability = 0.1
Milwaukee Brewers

WS Probability = 0.1
Kansas City Royals

WS Probability = 0.09
Oakland Athletics

WS Probability = 0.09
San Diego Padres

WS Probability = 0.08
Baltimore Orioles

WS Probability = 0.08
Toronto Blue Jays

WS Probability = 0.08
Cleveland Indians

WS Probability = 0.07
Detroit Tigers

WS Probability = 0.07
Arizona Diamondbacks

WS Probability = 0.07
Philadelphia Phillies

WS Probability = 0.06
Colorado Rockies

WS Probability = 0.05
Texas Rangers

WS Probability = 0.04
New York Mets

WS Probability = 0.04
Miami Marlins

WS Probability = 0.03
Cincinnati Reds


In [25]:
predict_the_winner(model_12, 2016, team_data)


WS Probability = 0.28
Los Angeles Dodgers

WS Probability = 0.22
Houston Astros

WS Probability = 0.21
New York Yankees

WS Probability = 0.15
Chicago Cubs

WS Probability = 0.14
Seattle Mariners

WS Probability = 0.12
Minnesota Twins

WS Probability = 0.11
Cleveland Indians

WS Probability = 0.1
Texas Rangers

WS Probability = 0.1
Los Angeles Angels

WS Probability = 0.09
Boston Red Sox

WS Probability = 0.08
Toronto Blue Jays

WS Probability = 0.08
Kansas City Royals

WS Probability = 0.07
Detroit Tigers

WS Probability = 0.07
Arizona Diamondbacks

WS Probability = 0.07
Baltimore Orioles

WS Probability = 0.06
Milwaukee Brewers

WS Probability = 0.06
Tampa Bay Rays

WS Probability = 0.06
Miami Marlins

WS Probability = 0.06
Washington Nationals

WS Probability = 0.05
San Francisco Giants

WS Probability = 0.03
Colorado Rockies

WS Probability = 0.03
Oakland Athletics

WS Probability = 0.02
Philadelphia Phillies

WS Probability = 0.02
Cincinnati Reds

WS Probability = 0.01
New York M

In [26]:
predict_the_winner(model_6, 2016, team_data)


WS Probability = 0.29
Houston Astros

WS Probability = 0.28
Los Angeles Dodgers

WS Probability = 0.21
New York Yankees

WS Probability = 0.19
Cleveland Indians

WS Probability = 0.16
Chicago Cubs

WS Probability = 0.13
Washington Nationals

WS Probability = 0.12
Arizona Diamondbacks

WS Probability = 0.12
Boston Red Sox

WS Probability = 0.11
Tampa Bay Rays

WS Probability = 0.09
Seattle Mariners

WS Probability = 0.09
Toronto Blue Jays

WS Probability = 0.08
Milwaukee Brewers

WS Probability = 0.08
Baltimore Orioles

WS Probability = 0.07
San Diego Padres

WS Probability = 0.06
Colorado Rockies

WS Probability = 0.06
New York Mets

WS Probability = 0.06
Kansas City Royals

WS Probability = 0.05
San Francisco Giants

WS Probability = 0.05
Minnesota Twins

WS Probability = 0.04
Los Angeles Angels

WS Probability = 0.04
Atlanta Braves

WS Probability = 0.03
Texas Rangers

WS Probability = 0.03
Miami Marlins

WS Probability = 0.03
Oakland Athletics

WS Probability = 0.02
Chicago White S

In [27]:
predict_the_winner(model_3, 2016, team_data)


WS Probability = 0.26
Houston Astros

WS Probability = 0.25
Los Angeles Dodgers

WS Probability = 0.18
Cleveland Indians

WS Probability = 0.16
Boston Red Sox

WS Probability = 0.15
New York Yankees

WS Probability = 0.13
Arizona Diamondbacks

WS Probability = 0.13
Tampa Bay Rays

WS Probability = 0.11
Toronto Blue Jays

WS Probability = 0.11
Washington Nationals

WS Probability = 0.11
Los Angeles Angels

WS Probability = 0.09
Minnesota Twins

WS Probability = 0.07
Chicago Cubs

WS Probability = 0.07
Seattle Mariners

WS Probability = 0.07
Baltimore Orioles

WS Probability = 0.06
Kansas City Royals

WS Probability = 0.05
San Francisco Giants

WS Probability = 0.04
Milwaukee Brewers

WS Probability = 0.04
New York Mets

WS Probability = 0.04
Pittsburgh Pirates

WS Probability = 0.03
Chicago White Sox

WS Probability = 0.03
Texas Rangers

WS Probability = 0.03
Oakland Athletics

WS Probability = 0.03
St. Louis Cardinals

WS Probability = 0.03
Philadelphia Phillies

WS Probability = 0.02

not sure.  move on.